# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [28]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [29]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_new csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [2]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Completing the Apache Cassandra coding portion of the project. 

## Now we're ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [2]:
# This should make a connection to a Cassandra instance on your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [3]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project_1b 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [4]:
try:
    session.set_keyspace('project_1b')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries by modeling the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


    

## Query 1:

### How should we model this data? What should be our Primary Key and Partition Key? 

#### Since we're looking for data based off the session_id let's set the partition key as the `sessionId`. Then the clustering column can be `item_in_session`, as we're also filtering based on that. Since every item in a specific `session_id` should only occur once this will set our rows to be unique.

### Create Tables

In [13]:
query = "CREATE TABLE IF NOT EXISTS song_details_library "
query = query + "(session_id int, item_in_session int, artist text, song text, length float, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Insert the data into the table



In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_details_library (session_id, item_in_session, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

### Do a SELECT to verify that the data has been inserted into each table

In [15]:
query = "select artist, song, length from song_details_library WHERE session_id=338 and item_in_session=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

song_details_df = pd.DataFrame([rows[0]], columns=['artist_name', 'song_title', 'song_length'])
song_details_df.head()

,artist_name,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


## Query 2:

### How should we model this data? What should be our Primary Key and Partition Key? 

#### Since we're looking for data based off the user_id let's set the partition key as `user_id`. Then the clustering columns can be `session_id`, as we're also filtering based on that. Then since we need to sort songs by `item_in_session` lets set that as the third column in the clustering columns.

### Create Tables

In [7]:
query = "CREATE TABLE IF NOT EXISTS user_song_library "
query = query + "(user_id int, session_id int, item_in_session int, artist text, song text, user_lname text, \
                    user_fname text, PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)        

### Insert the data into the table


In [8]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_song_library (user_id, session_id, item_in_session, artist, song, user_lname, user_fname)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[4], line[1]))


### Do a SELECT to verify that the data has been inserted into each table

In [9]:
query = "select artist, song, user_fname, user_lname from user_song_library WHERE user_id=10 and session_id=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

rows_data = [list(x) for x in rows]
    
user_song_df = pd.DataFrame(rows_data, columns=['artist_name', 'song_title', 'user_fname', 'user_lname'])
user_song_df.head()

,artist_name,song_title,user_fname,user_lname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


## Query 3:

### How should we model this data? What should be our Primary Key and Partition Key? 

#### Since we're looking for data based off the song let's set the partition key as `song`. Then to ensure uniqueness let's add `user_id` because we want to get the unique users who listened to the song "All Hands Against His Own". We shouldn't use `user_lname` and `user_fname` as clustering columns because there could be multiple users with the same name.


### Create Tables

In [19]:
query = "CREATE TABLE IF NOT EXISTS users_for_song "
query = query + "(song text, user_id int, user_lname text, user_fname text, PRIMARY KEY (song, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)        
                    

### Insert the data into the table


In [20]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_for_song (song, user_id, user_lname, user_fname)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[4], line[1]))

#### Do a SELECT to verify that the data has been inserted into each table

In [21]:
query = "select user_fname, user_lname from users_for_song WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

rows_data = [list(x) for x in rows]

users_for_song_df = pd.DataFrame(rows_data, columns=['user_fname', 'user_lname'])
users_for_song_df.head()

,user_fname,user_lname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [ ]:
query = "drop table song_details_library"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table user_song_library"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "drop table users_for_song"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [11]:
session.shutdown()
cluster.shutdown()